# Building A Crime Reports Database: PostgreSQL.

In this project we will be building a database for crime reports in boston. The data is available to us in form of a csv file, **boston.csv**.

We will save this data in a table named boston_crimes which will be located in a schema named crimes. We shall name our database crime_db.

We will also create two groups **readonly** and **readwrite** with appropriate privileges. And we shall then create a user for each of the above-mentioned groups.

Below are the first few rows of our data.

In [1]:
import pandas as pd
data = pd.read_csv('boston.csv')
# data.head()
data['incident_number']

0              1
1              2
2              3
3              4
4              5
           ...  
298324    298325
298325    298326
298326    298327
298327    298328
298328    298329
Name: incident_number, Length: 298329, dtype: int64

Below is an illustration of what we intend to acheive: at a higher level.
![work overview](https://dq-content.s3.amazonaws.com/250/goal.png)

## The Crime Database
We shall now create the crime database and name it **crime_db**. Also we shall create a schema called **crimes**

In [2]:
import psycopg2
#creating our database
conn = psycopg2.connect(dbname='dq', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE DATABASE crime_db;')
conn.close()

DuplicateDatabase: database "crime_db" already exists


In [3]:
#creating our schema
conn = psycopg2.connect(dbname='crime_db', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE SCHEMA crimes;')

DuplicateSchema: schema "crimes" already exists


## Obtaining The Column Names And Sample

Let's now gather some data about our crime dataset. That way we shall be able to easily select the right datatypes to use in our table.


In [4]:
#open the boston.csv file using context manager
import csv
with open('boston.csv','r') as f:
    reader = csv.reader(f)
    rows = list(reader)
    col_headers =rows[0]
    first_row = rows[1]
    
print(col_headers, first_row, sep='\n' )

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

This returns a set of all distinct values contained in a column. 

This function will help us to check whether the enumerated datatype might be a good choice for representing a certain columns. 

The function will also help us in computing the maximum length of any text-like column - hence we will be able to select the appropriate sizes for VARCHAR columns if need be.

In [5]:
def get_col_value_set(csv_filename, col_index):
    with open(csv_filename,'r') as f:
        next(f)
        reader = csv.reader(f)
        rows = list(reader)
        set_distinct = set()
        for row in rows:
            if row[col_index] not in set_distinct:
                set_distinct.add(row[col_index])
    return set_distinct
           

In [6]:
get_col_value_set("boston.csv", 4)

{'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'}

In [7]:
#we can compute the number of distinct values for each column.
distinct_values_dic = {}
for i in range(6):
    col_index = i
    column_distinct_values = get_col_value_set("boston.csv", col_index)
    number_distinct_values = len(column_distinct_values)
    distinct_values_dic[col_headers[col_index]] = number_distinct_values
    

print(distinct_values_dic)

{'incident_number': 298329, 'offense_code': 219, 'description': 239, 'date': 1177, 'day_of_the_week': 7, 'lat': 18177}


## Finding The Maximum Length

In [8]:
print(col_headers.index('description'))
print(col_headers.index('day_of_the_week'))

2
4


In [9]:
#Using the function get_col_value_set(), 
#compute the maximum length of any value in the description column
x = col_headers.index('description') #index for description column
col_values = get_col_value_set("boston.csv", x)#values in description
max_len = 0
for i in col_values:
    max_len = max(max_len,len(i))
print('The maximum length of any value in the description column is', max_len)

The maximum length of any value in the description column is 58


In [10]:
col_values

{'A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.',
 'A&B ON POLICE OFFICER',
 'ABDUCTION - INTICING',
 'AFFRAY',
 'AIRCRAFT INCIDENTS',
 'ANIMAL ABUSE',
 'ANIMAL CONTROL - DOG BITES - ETC.',
 'ANIMAL INCIDENTS',
 'ANNOYING AND ACCOSTIN',
 'ANNOYING AND ACCOSTING',
 'ARSON',
 'ASSAULT & BATTERY',
 'ASSAULT & BATTERY D/W - KNIFE',
 'ASSAULT & BATTERY D/W - OTHER',
 'ASSAULT & BATTERY D/W - OTHER ON POLICE OFFICER',
 'ASSAULT - AGGRAVATED',
 'ASSAULT - AGGRAVATED - BATTERY',
 'ASSAULT - SIMPLE',
 'ASSAULT D/W - KNIFE ON POLICE OFFICER',
 'ASSAULT D/W - OTHER',
 'ASSAULT SIMPLE - BATTERY',
 'AUTO THEFT',
 'AUTO THEFT - LEASED/RENTED VEHICLE',
 'AUTO THEFT - MOTORCYCLE / SCOOTER',
 'AUTO THEFT - OUTSIDE - RECOVERED IN BOSTON',
 'AUTO THEFT - RECOVERED IN BY POLICE',
 'AUTO THEFT LEASE/RENT VEHICLE',
 'AUTO THEFT OTHER',
 'B&E NON-RESIDENCE DAY - FORCIBLE',
 'B&E NON-RESIDENCE DAY - NO FORCE',
 'B&E NON-RESIDENCE DAY - NO PROP TAKEN',
 'B&E NON-RESIDENCE NIGHT - FORCE',
 'B&E RESIDENCE DAY - N

## Creating The Table

In [11]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


I have chosen to use the following datatypes for the various columns in the boston crimes dataset. 
- incident_number: integer
- offense_code: integer
- description: VARCHAR(60)
- date: datetime
- day_of_the_week : enumerate
- lat: decimal
- long: decimal

In [12]:
#creating an enumerated datatype for the day_of_the_week column
weekday_tuple = tuple(get_col_value_set("boston.csv", 4))
cur.execute('''
    CREATE TYPE weekday  
    AS ENUM (%s,%s,%s,%s,%s,%s,%s);
''',weekday_tuple)

DuplicateObject: type "weekday" already exists


In [13]:
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [14]:
#creating a table called crimes.boston_crimes
cur.execute('''
    CREATE TABLE crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(60),
    date DATE,
    day_of_the_week weekday,
    lat decimal,
    long decimal
    );
''')


DuplicateTable: relation "boston_crimes" already exists


## Loading The Data

In [15]:
with open('boston.csv','r') as f:
    cur.copy_expert('''
        COPY crimes.boston_crimes 
        FROM STDIN WITH CSV HEADER
    ''',f)

UniqueViolation: duplicate key value violates unique constraint "boston_crimes_pkey"
DETAIL:  Key (incident_number)=(1) already exists.
CONTEXT:  COPY boston_crimes, line 2


In [17]:
cur.execute('SELECT * FROM crimes.boston_crimes;')
len(cur.fetchall())

298329

From above we can see that we have all the data now loaded in our newly created table.

Now our database should look like so:
![database after loading data](https://dq-content.s3.amazonaws.com/250/table_created.png)

## Revoking Public Privileges

In [18]:
cur.execute('REVOKE ALL ON SCHEMA public FROM public;')

In [19]:
cur.execute('REVOKE ALL ON DATABASE crime_db FROM public;')

After revoking all privileges from the public group on the database and on the public schema, we can now go ahead and create our two user groups: **readonly** and **readwrite**.

## Creating User Groups

In [24]:
#creating the readonly group
cur.execute('''
    CREATE GROUP readonly NOLOGIN;
''')

#creating the readwrite group
cur.execute('''
    CREATE GROUP readwrite NOLOGIN;
''')

#granting CONNECT privileges
cur.execute('''
    GRANT CONNECT ON DATABASE crime_db TO readonly;
''')
cur.execute('''
    GRANT CONNECT ON DATABASE crime_db TO readwrite;
''')

#granting USAGE privileges
cur.execute('''
    GRANT USAGE ON SCHEMA crimes TO readonly;
''')
cur.execute('''
    GRANT USAGE ON SCHEMA crimes TO readwrite;
''')

#granting group specific privileges on all tables in the crimes
#schema
cur.execute('''
    GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;
''')
cur.execute('''
    GRANT SELECT, INSERT, DELETE, UPDATE 
    ON ALL TABLES IN SCHEMA crimes TO readwrite;
''')

DuplicateObject: role "readonly" already exists


At the moment a high-level picture of our database looks like so:
![database after creating user groups](https://dq-content.s3.amazonaws.com/250/user_groups.png)

## Creating Users

In [28]:
#creating users data_analyst and data_scientist
cur.execute('''
    CREATE USER data_analyst WITH PASSWORD 'secret1';
''')
cur.execute('''
    CREATE USER data_scientist WITH PASSWORD 'secret2';
''')

#assigning the users above to groups
cur.execute('''
    GRANT readonly TO data_analyst;
''')
cur.execute('''
    GRANT readwrite TO data_scientist;
''')

InvalidGrantOperation: role "readwrite" is a member of role "data_scientist"
